
# Assignment 4 for Course 1MS041
Make sure you pass the `# ... Test` cells and
 submit your solution notebook in the corresponding assignment on the course website. You can submit multiple times before the deadline and your highest score will be used.

---
## Assignment 4, PROBLEM 1
Maximum Points = 24


    This time the assignment only consists of one problem, but we will do a more comprehensive analysis instead.

Consider the dataset `Corona_NLP_train.csv` that you can get from the course website [git](https://github.com/datascience-intro/1MS041-2024/blob/main/notebooks/data/Corona_NLP_train.csv). The data is "Coronavirus tweets NLP - Text Classification" that can be found on [kaggle](https://www.kaggle.com/datasets/datatattle/covid-19-nlp-text-classification). The data has several columns, but we will only be working with `OriginalTweet`and `Sentiment`.

1. [3p] Load the data and filter out those tweets that have `Sentiment`=`Neutral`. Let $X$ represent the `OriginalTweet` and let 
    $$
        Y = 
        \begin{cases}
        1 & \text{if sentiment is towards positive}
        \\
        0 & \text{if sentiment is towards negative}.
        \end{cases}
    $$
    Put the resulting arrays into the variables $X$ and $Y$. Split the data into three parts, train/test/validation where train is 60% of the data, test is 15% and validation is 25% of the data. Do not do this randomly, this is to make sure that we all did the same splits (we are in this case assuming the data is IID as presented in the dataset). That is [train,test,validation] is the splitting layout.

2. [4p] There are many ways to solve this classification problem. The first main issue to resolve is to convert the $X$ variable to something that you can feed into a machine learning model. For instance, you can first use [`CountVectorizer`](https://scikit-learn.org/1.5/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) as the first step. The step that comes after should be a `LogisticRegression` model, but for this to work you need to put together the `CountVectorizer` and the `LogisticRegression` model into a [`Pipeline`](https://scikit-learn.org/1.5/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline). Fill in the variable `model` such that it accepts the raw text as input and outputs a number $0$ or $1$, make sure that `model.predict_proba` works for this. **Hint: You might need to play with the parameters of LogisticRegression to get convergence, make sure that it doesn't take too long or the autograder might kill your code**
3. [3p] Use your trained model and calculate the precision and recall on both classes. Fill in the corresponding variables with the answer.
4. [3p] Let us now define a cost function
    * A positive tweet that is classified as negative will have a cost of 1
    * A negative tweet that is classified as positive will have a cost of 5
    * Correct classifications cost 0
    
    complete filling the function `cost` to compute the cost of a prediction model under a certain prediction threshold (recall our precision recall lecture and the `predict_proba` function from trained models). 

5. [4p] Now, we wish to select the threshold of our classifier that minimizes the cost, fill in the selected threshold value in value `optimal_threshold`.
6. [4p] With your newly computed threshold value, compute the cost of putting this model in production by computing the cost using the validation data. Also provide a confidence interval of the cost using Hoeffdings inequality with a 99% confidence.
7. [3p] Let $t$ be the threshold you found and $f$ the model you fitted (one of the outputs of `predict_proba`), if we define the random variable
    $$
        C = (1-1_{f(X)\geq t})Y+5(1-Y)1_{f(X) \geq t}
    $$
    then $C$ denotes the cost of a randomly chosen tweet. In the previous step we estimated $\mathbb{E}[C]$ using the empirical mean. However, since the threshold is chosen to minimize cost it is likely that $C=0$ or $C=1$ than $C=5$ as such it will have a low variance. Compute the empirical variance of $C$ on the validation set. What would be the confidence interval if we used Bennett's inequality instead of Hoeffding in point 6 but with the computed empirical variance as our guess for the variance?

In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
# Part 1

# Load the data from the file specified in the problem definition and make sure that it is loaded using
# the search path `data/Corona_NLP_train.csv`. This is to make sure the autograder and your computer have the same
# file path and can load the data correctly.

# Contrary to how many other problems are structured, this problem actually requires you to
# have X on the shape (n_samples, ) that is a 1-dimensional array. Otherwise it will cause a bunch
# of errors in the autograder or also in for instance CountVectorizer.

# Make sure that all your data is numpy arrays and not pandas dataframes or series.

#get the csv file and read it into a pandas dataframe
df  = pd.read_csv("data/Corona_NLP_train.csv",  encoding= "ISO-8859-1")
#remove all columns except OriginalTweet and Sentiment
df = df.drop(columns=["UserName","ScreenName","Location","TweetAt"])
#dropping the rows that have a neutral sentiment
df = df.drop(index = df.loc[df["Sentiment"] == "Neutral"].index)
#mapping the sentiment as 1 or 0
df["Sentiment"] = df["Sentiment"].map({"Positive": 1, "Extremely Positive": 1, "Negative": 0, "Extremely Negative":0})

X = np.array(df["OriginalTweet"].tolist())
Y = np.array(df["Sentiment"].tolist())
length = X.shape[0]

X_train = X[0:round(length*.6)]
Y_train = Y[0:round(length*.6)]
X_test = X[round(length*.6):round(length*.75)-1]
Y_test = Y[round(length*.6):round(length*.75)-1]
X_valid = X[round(length*.75)-1:]
Y_valid = Y[round(length*.75)-1:]

# print(len(X_train)/length)
# print(len(Y_train)/length)
# print(len(X_test)/length)
# print(len(Y_test)/length)
# print(len(X_valid)/length)
# print(len(Y_valid)/length)
# print(X_train)
# print(df)
#print(len(X_train) + len(X_test) + len(X_valid))
# print(X_train.shape)
# print(X_test.shape)
# print(X_valid.shape)

In [2]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
# Part 2

# Train a machine learning model or pipeline that can take the raw strings from X and predict Y=0,1 depending on the
# sentiment of the tweet. Store the trained model in the variable `model`.


model = Pipeline([("Vect",CountVectorizer()),("Logistic Regression", LogisticRegression(max_iter=10000))])
model.fit(X_train,Y_train)
# print(model.predict_proba(X_train).shape)

Pipeline(steps=[('Vect', CountVectorizer()),
                ('Logistic Regression', LogisticRegression(max_iter=10000))])

In [3]:

# Part 3
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
# Evaluate the model on the test set and calculate precision, and recall on both classes. Store the results in the
# variables `precision_0`, `precision_1`, `recall_0`, `recall_1`.

#get our predictions 
y_pred = model.predict(X_test)

precision_0 = precision_score(Y_test,y_pred,pos_label=0)
precision_1 = precision_score(Y_test, y_pred)
recall_0 = recall_score(Y_test,y_pred,pos_label=0)
recall_1 = recall_score(Y_test,y_pred)

# print(precision_0)
# print(precision_1)
# print(recall_0)
# print(recall_1)

In [4]:

# Part 4

def cost(model,threshold,X,Y):
    # Hint, make sure that the model has a predict_proba method
    # think about how the decision is made based on the probabilities
    # and how the threshold can be used to make the decision.
    # For reference take a look at the lecture notes "Bayes classifier"
    # which contains how the decision is made based on the probabilities when the threshold is 0.5.
    
    # Fill in what is missing to compute the cost and return it
    # Note that we are interested in average cost
    #get all our predicitons 
    predictions = model.predict_proba(X)
    # print(len(predictions))
    #total cost
    cost = 0
    #classifying every prediction
    for i in range(len(predictions)):
        prediction = None
        #if our prediction of the positive class is above the threshold we will classify the guess as a 1
        #otherwise we will classify it as a 0
        if predictions[i][1] > threshold:
            prediction = 1
        else:
            prediction = 0
        
        #a psoitive tweet that is classified as negative add 1
        if Y[i] == 1 and prediction == 0:
            cost += 1
        # a negative tweet that is classified as positive
        if Y[i] == 0 and prediction == 1:
            cost += 5

    # print("Sum of cost: ", cost)
    # print("Length of predictions: ", len(predictions))
    #return the average cost
    return cost/ len(predictions)

# cost(model, 0.5, X_test,Y_test)

In [5]:

# Part 5

# Find the optimal threshold for the model on the test set. Store the threshold in the variable `optimal_threshold`
# and the cost at the optimal threshold in the variable `cost_at_optimal_threshold` evaluated on the test set.

min_cost = 100
threshold = -1
for i in range(0,100):
    c = cost(model,i/100,X_test,Y_test)
    if c < min_cost:
        min_cost = c
        threshold = i/100

# print(threshold)
# print(min_cost)
# print(cost(model,.8,X_test,Y_test))
# print(cost(model,.7,X_test,Y_test))
optimal_threshold = threshold
cost_at_optimal_threshold = min_cost

In [14]:

# Part 6
# cost_interval_valid = ()
cost_at_optimal_threshold_valid = cost(model,optimal_threshold,X_valid,Y_valid)
# print(cost_at_optimal_threshold_valid)
#print(cost_at_optimal_threshold_valid
alpha = 0.01
def epsilon_bounded(n,b,alpha):
    """
    Calculates the epsilon for P(|X - E[X]| >= epsilon) < alpha
    using Hoeffdings inequality.

    Parameters
    ----------
    n : the number of samples
    b : |X| <= b
    alpha : the significance level

    Returns
    -----------
    epsilon
    """
    import numpy as np
    return b*np.sqrt(-1/(2*n)*np.log((alpha)/2))

epsilon_hoff_benny = epsilon_bounded(X_valid.shape[0], 1, alpha)
print("Epsilon Benny: ", epsilon_hoff_benny)
epsilon_hoff = np.sqrt((1/(2*X_valid.shape[0]))*np.log(2/alpha))
print(epsilon_hoff)
cost_interval_valid = (cost_at_optimal_threshold_valid - epsilon_hoff_benny,cost_at_optimal_threshold_valid + epsilon_hoff_benny)
print(cost_at_optimal_threshold_valid)
print(cost_interval_valid)

assert(type(cost_interval_valid) == tuple)
assert(len(cost_interval_valid) == 2)

Epsilon Benny:  0.017799135323997965
0.017799135323997965
0.27971777086821337
(0.2619186355442154, 0.29751690619221133)


In [8]:

# Part 7

predictions_7 = model.predict_proba(X_valid)
#total cost
c = []
    #classifying every prediction
for i in range(len(predictions_7)):
    prediction_7 = None
    #if our prediction of the positive class is above the threshold we will classify the guess as a 1
    #otherwise we will classify it as a 0
    if predictions_7[i][1] > optimal_threshold:
        prediction_7 = 1
    else:
        prediction_7 = 0
    
    #a psoitive tweet that is classified as negative add 1
    if Y_valid[i] == 1 and prediction_7 == 0:
        c.append(1)
    # a negative tweet that is classified as positive
    elif Y_valid[i] == 0 and prediction_7 == 1:
        c.append(5)
    else:
        c.append(0)
c = np.array(c)
# print("Sum of cost: ", c.sum())
# print("Length of predictions: ", len(predictions_7))
variance_of_C = np.var(c, ddof= 1)
print(variance_of_C)

def bennett_epsilon(n,b,sigma,alpha):
    """
    Calculates the epsilon for P(|X - E[X]| >= epsilon) < alpha
    using Bennets inequality.

    It also prints out the approximation error in inverting the function in Bennets inequality

    Parameters
    ----------
    n : the number of samples
    b : |X| <= b
    sigma : the standard deviation of X
    alpha : the significance level

    Returns
    -----------
    epsilon
    """
    import scipy.optimize as so

    h = lambda u: (1+u)*np.log(1+u)-u
    f = lambda epsilon: np.exp(-n*sigma**2/b**2*h(b*epsilon/sigma**2))-alpha/2
    ans = so.fsolve(f,0.002)
    epsilon = np.abs(ans[0])
    # print("Numerical error", f(epsilon))
    return epsilon

alpha = 0.01
epsilon_ben = bennett_epsilon(len(X_valid),1, np.sqrt(variance_of_C),alpha)
# print(epsilon_ben)
interval_of_C = (cost_at_optimal_threshold_valid - epsilon_ben, cost_at_optimal_threshold_valid + epsilon_ben)
print(interval_of_C)
assert(type(interval_of_C) == tuple)
assert(len(interval_of_C) == 2)

0.7062241518884402
(0.2495915321331834, 0.3098440096032433)
